In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard library imports.
import itertools

# Related third party imports.
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation
import src.data.graph_utilities as gu

In [4]:
network_folder = '../data/processed/networks'
network_name = '26_usa'
factor = 0.5

# Load data
g = nx.read_gml(f'{network_folder}/{network_name}.gml', label="id")
diameter = gu.get_graph_diameter(g)
ml = diameter * factor

ns = NetworkSimulation(network_name, **{'max_length':ml, 'shortest_k':16})
S, H, C = list(g.nodes), list(g.nodes), list(g.nodes)
H_pairs = list(itertools.combinations_with_replacement(H,2))
HS_pairs = list(itertools.product(H, S))
HHS_pairs = list(itertools.product(H_pairs, S))
CS_pairs = list(itertools.product(C, S))

allowed_switch_H_pairs = gu.get_allowed_hypervisor_pairs_by_switch(ns.network_operator.triplets_by_switches)
# not_allowed_switch_H_pairs = {s:[(h1,h2) for h1, h2 in H_pairs if (h1,h2) not in allowed_switch_H_pairs[s]] for s in S}
# switch_H_pairs = {s:[(i,j) for _,i,j in ns.network_operator.triplets_by_switches[s] if (i!=s and j!=s)] for s in S}


In [8]:
model = gp.Model("mip1")

active_hypervisors = model.addVars(H, vtype=GRB.BINARY)
hypervisor_controls_switch = model.addVars(HS_pairs, vtype=GRB.BINARY)
hypervisor_pair_controls_switch = model.addVars(HHS_pairs, vtype=GRB.BINARY)

# Only active hypervisors can control switches, Hypervisors without controlled switches are inactive
c_1 = model.addConstrs(active_hypervisors[h] == gp.or_([hypervisor_controls_switch[(h,s)] for s in S]) for h in H)

# Each switch is controlled by a pair of hypervisors except when there is a hypervisor at the switch’s location
c_2a = model.addConstrs(active_hypervisors[s] <= hypervisor_controls_switch[(s,s)] for s in S)
# c_2a = model.addConstrs(active_hypervisors[s] <= hypervisor_pair_controls_switch[((s,s),s)] for s in S)
c_2b = model.addConstrs(hypervisor_controls_switch[(s,s)] + gp.LinExpr([(1, hypervisor_controls_switch[(h,s)]) for h in H]) == 2 for s in S)
# c_2b = model.addConstrs(gp.quicksum([hypervisor_pair_controls_switch[((h1,h2),s)] for h1,h2 in H_pairs]) == 1 for s in S)

# The hypervisor pair controls the switch if both of them are controlling it
c_3 = model.addConstrs(hypervisor_pair_controls_switch[((h1,h2),s)] == gp.and_(hypervisor_controls_switch[(h1,s)], hypervisor_controls_switch[(h2,s)]) for (h1,h2),s in HHS_pairs if not (h1==h2 and h1!=s))

# Only valid triplets (T) can appear
c_4 = model.addConstrs(gp.quicksum([hypervisor_pair_controls_switch[((h1,h2),s)] for h1,h2 in allowed_switch_H_pairs[s]]) == 1 for s in S)
# c_5a = model.addConstrs(gp.quicksum([hypervisor_pair_controls_switch[((h1,h2),s)] for h1,h2 in not_allowed_switch_H_pairs[s]]) == 0 for s in S)
# c_5b = model.addConstrs(hypervisor_pair_controls_switch[((h1,h2),s)] == 0 for h1,h2 in not_allowed_switch_H_pairs[s] for s in S)
# c_5c = model.addConstrs(hypervisor_controls_switch[(h1,s)] + hypervisor_controls_switch[(h2,s)] <= 1 for h1,h2 in not_allowed_switch_H_pairs[s] for s in S)

# c_6 = model.addConstr(gp.quicksum(active_hypervisors) <= 5)

# Minimize the number of hypervisors
model.setObjective(gp.quicksum(active_hypervisors), GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 78 rows, 9828 columns and 2827 nonzeros
Model fingerprint: 0x74b20d29
Model has 8502 general constraints
Variable types: 0 continuous, 9828 integer (9828 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve added 3090 rows and 0 columns
Presolve removed 0 rows and 7079 columns
Presolve time: 0.10s
Presolved: 3168 rows, 2749 columns, 14833 nonzeros
Variable types: 0 continuous, 2749 integer (2749 binary)
Found heuristic solution: objective 25.0000000

Root relaxation: objective 3.666667e+00, 1964 iterations, 0.12 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.66667    0  120   25.0000

In [6]:
for h,v in active_hypervisors.items():
    if v.x > 0.9:
        print('%s' % (h))
print('Obj: %g' % model.objVal)

2
6
10
21
Obj: 4


In [15]:
for ((h1,h2),s),v in hypervisor_pair_controls_switch.items():
    if v.x > 0.9:
        print('%2s %2s %2s' % (s, h1, h2))

 4  4  4
 6  4  7
 3  4 11
 7  7  7
 9  7 10
22  7 10
 8  8  8
10 10 10
11 11 11
12 11 13
 5 11 25
13 13 13
14 14 14
15 15 15
 0 15 19
 1 15 19
 2 15 19
16 15 19
17 15 19
18 15 19
20 15 19
21 15 19
24 15 19
19 19 19
23 23 23
25 25 25
